En este Jupyter Notebook construiremos un modelo que nos permita predecir quién ganará una carrera y en qué posición quedará un piloto

In [100]:
import sys

sys.path.append('../')

from src.race_models_support import *

In [1]:
import pandas as pd
import numpy as np
import requests

---

Esto veremos

In [ ]:
season = 2023

url = f'http://api.jolpi.ca/ergast/f1/{season}/driverstandings/'

response = requests.get(url)
print(response.status_code)

content = response.json()

Obtener los circuitos de cada temporada

In [20]:
from fastf1.ergast import Ergast
import fastf1

ergast = Ergast()

---

Vamos a obtener los datos para crear el modelo predictivo

Cargamos los resultados de una sesión (carrera). No necesitamos telemetría ni tiempo atmosférico

In [101]:
session = fastf1.get_session(2014, 1, 'R')



In [108]:
session.results

DataNotLoadedError: The data you are trying to access has not been loaded yet. See `Session.load`

In [15]:
def get_race_results(session, margin = 100):
    """
    Retrieves race results and processes the data to standardize timing and statuses.

    Parameters
    -----------
    - session (fastf1.core.Session): The FastF1 session object containing race data, including results.
    - margin (int, optional): The time margin (in seconds) added to non-finishers' times. Defaults to 100.

    Returns
    --------
    - (pd.DataFrame): A DataFrame containing processed race results with the following columns:
        - 'DriverId': Identifier for the driver.
        - 'TeamId': Identifier for the team.
        - 'Position': Final position of the driver.
        - 'GridPosition': Starting grid position of the driver.
        - 'Time': Race time in seconds (adjusted for winners and non-finishers).
        - 'Status': Status of the driver (e.g., 'Finished', 'Retired').
        - 'Points': Points scored by the driver.
    """

    # Get results dataframe
    results = session.results
    results = results.loc[:, ['DriverId', 'TeamId', 'Position', 'GridPosition', 'Time', 'Status', 'Points']]

    # Fix winner time to 0 and convert to seconds
    results.iloc[0, results.columns.get_loc('Time')] = pd.Timedelta(0)
    results['Time'] = results['Time'].dt.total_seconds()

    # Fix non-finishers time to avoid NaT
    max_finished_time = results.loc[results['Status'] == 'Finished', 'Time'].max()
    results.loc[results['Status'] != 'Finished', 'Time'] = max_finished_time + margin

    return results

    """
    # Weather
    compounds = session.laps['Compound'].value_counts(normalize=True)
    results['weather'] = get_weather_condition(compounds)

    # Flags, SC and VSC
    track_status = session.track_status['Message'].value_counts()
    results['yellows'] = track_status.get('Yellow', 0)
    results['reds'] = track_status.get('Red', 0)
    results['safety_cars'] = track_status.get('SCDeployed', 0)
    results['virtual_safety_cars'] = track_status.get('VSCDeployed', 0)
    """

    return results


In [ ]:
def get_info(session, info):
    # Validación de entrada
    if not all(key in info for key in ['weather', 'yellows', 'reds', 'sc', 'vsc']):
        raise ValueError("El diccionario 'info' debe contener las claves: 'weather', 'yellows', 'reds', 'sc', 'vsc'")
    
    # Validar que las columnas necesarias existen
    if 'Compound' not in session.laps.columns or 'Message' not in session.track_status.columns:
        raise KeyError("Las columnas 'Compound' o 'Message' no están disponibles en los datos de la sesión.")
    
    # Procesamiento en un solo paso
    compounds = session.laps['Compound'].value_counts(normalize=True)
    track_status_counts = session.track_status['Message'].value_counts()

    # Extraer métricas
    info['weather'].append(get_weather_condition(compounds))
    info['yellows'].append(track_status_counts.get('Yellow', 0))
    info['reds'].append(track_status_counts.get('Red', 0))
    info['sc'].append(track_status_counts.get('SCDeployed', 0))
    info['vsc'].append(track_status_counts.get('VSCDeployed', 0))


In [75]:
def get_info(session, info):

    # Weather
    compounds = session.laps['Compound'].value_counts(normalize=True)
    info['weather'].append(get_weather_condition(compounds))

    # Flags, SC and VSC
    track_status = session.track_status['Message'].value_counts()
    info['yellows'].append(track_status.get('Yellow', 0))
    info['reds'].append(track_status.get('Red', 0))
    info['sc'].append(track_status.get('SCDeployed', 0))
    info['vsc'].append(track_status.get('VSCDeployed', 0))

In [115]:
from tqdm import tqdm
from time import sleep

In [77]:
results_df = pd.DataFrame()
races_df = pd.DataFrame()

for i in range(2020, 2021):

    # Obtener las carreras de la temporada
    races = ergast.get_race_schedule(i)
    races = races.loc[:, ['season', 'round', 'circuitId']]

    # Añadimos a parte la información adicional
    info = {'weather': [], 'yellows': [], 'reds': [], 'sc': [], 'vsc': []}

    # Dentro de una temporada iteramos por carrera
    for race in tqdm(races.itertuples()):

        # Obtenemos la temporada y la ronda
        season = race[1]
        rnd = race[2]

        # Cargamos la sesión
        session = fastf1.get_session(season, rnd, 'R')
        session.load(telemetry=False, weather=False)

        # Obtenemos los resultados
        results = get_race_results(session)

        # Añadimos la información
        results['season'] = season
        results['round'] = rnd
        results['circuitId'] = race[3]

        # Juntamos con los resultados anteriores
        results_df = pd.concat([results_df, results])
        
        # La info adicional
        get_info(session, info)
        
        # break

    info_df = pd.DataFrame(info)
    
    # Añadimos la info
    races = pd.concat([races, info_df], axis=1)
    races_df = pd.concat([races_df, races])

    """
    # Ordenar para sumar bien
    results_df = results_df.sort_values(by='round', ascending=True)

    # Agrupa los resultados por piloto y calcula los puntos acumulados
    results_df['DriverPointsCumulative'] = results_df.groupby('DriverId')['Points'].cumsum()

    # Agrupa los resultados por equipo y calcula los puntos acumulados
    results_df['TeamPointsCumulative'] = results_df.groupby('TeamId')['Points'].cumsum()
    """


0it [00:00, ?it/s]logger      WARNING 	Failed to load schedule from FastF1 backend!
req            INFO 	No cached data found for season_schedule. Loading data...
_api           INFO 	Fetching season schedule...


1
[]
<class 'list'>


req            INFO 	Data has been written to cache!
core           INFO 	Loading data for Austrian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2020/1/results.json failed; using cached response
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/FinalProject/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/FinalProject/lib/python3.12/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/FinalProject/lib/python3.12/http/client.py", line 1428, in getresponse
    response.begin()
  File "/opt/homebrew/Caskroom/miniconda/bas

2
['dry']
<class 'list'>


core           INFO 	Loading data for Styrian Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
Request for URL https://ergast.com/api/f1/2020/2/results.json failed; using cached response
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/FinalProject/lib/python3.12/site-packages/urllib3/connectionpool.py", line 534, in _make_request
    response = conn.getresponse()
               ^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/FinalProject/lib/python3.12/site-packages/urllib3/connection.py", line 516, in getresponse
    httplib_response = super().getresponse()
                       ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Caskroom/miniconda/base/envs/FinalProject/lib/python3.12/http/client.py", line 1428, in getresponse
    response.begin()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/FinalProject/lib/python3.12/http/client.py", li

...

req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['44', '33', '23', '4', '55', '10', '3', '77', '31', '16', '26', '63', '5', '6', '7', '99', '20', '11', '18', '8']
15it [00:47,  2.24s/it]core           INFO 	Loading data for Sakhir Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


16
['dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'wet', 'dry']
<class 'list'>


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['11', '31', '18', '55', '3', '23', '26', '77', '63', '4', '10', '5', '99', '7', '20', '89', '51', '6', '33', '16']
16it [00:49,  2.16s/it]core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info


17
['dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'dry', 'wet', 'dry', 'dry']
<class 'list'>


req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	Driver  6: Lap timing integrity check failed for 1 lap(s)
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['33', '77', '44', '23', '4', '55', '3', '10', '31', '18', '26', '7', '16', '5', '63', '99', '6', '20', '51', '11']
17it [00:51,  3.01s/it]


In [82]:
df_aux = results_df.drop(columns=['weather', 'yellows', 'reds', 'safety_cars', 'virtual_safety_cars'])

In [98]:
df_aux.merge(right=races, how='inner', on=['season', 'round', 'circuitId'])

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,season,round,circuitId,weather,yellows,reds,sc,vsc
0,bottas,mercedes,1.0,1.0,0.000,Finished,25.0,2020,1,red_bull_ring,dry,7,0,3,0
1,leclerc,ferrari,2.0,7.0,2.700,Finished,18.0,2020,1,red_bull_ring,dry,7,0,3,0
2,norris,mclaren,3.0,3.0,5.491,Finished,16.0,2020,1,red_bull_ring,dry,7,0,3,0
3,hamilton,mercedes,4.0,5.0,5.689,Finished,12.0,2020,1,red_bull_ring,dry,7,0,3,0
4,sainz,mclaren,5.0,8.0,8.903,Finished,10.0,2020,1,red_bull_ring,dry,7,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,giovinazzi,alfa,16.0,14.0,202.738,+1 Lap,0.0,2020,17,yas_marina,dry,1,0,1,1
336,latifi,williams,17.0,18.0,202.738,+1 Lap,0.0,2020,17,yas_marina,dry,1,0,1,1
337,kevin_magnussen,haas,18.0,20.0,202.738,+1 Lap,0.0,2020,17,yas_marina,dry,1,0,1,1
338,pietro_fittipaldi,haas,19.0,17.0,202.738,+2 Laps,0.0,2020,17,yas_marina,dry,1,0,1,1


In [84]:
races

,season,round,circuitId,weather,yellows,reds,sc,vsc
0,2020,1,red_bull_ring,dry,7,0,3,0
1,2020,2,red_bull_ring,dry,1,0,1,0
2,2020,3,hungaroring,dry,4,0,0,0
3,2020,4,silverstone,dry,3,0,2,0
4,2020,5,silverstone,dry,2,0,0,0
5,2020,6,catalunya,dry,1,0,0,0
6,2020,7,spa,dry,4,0,1,0
7,2020,8,monza,dry,3,1,2,0
8,2020,9,mugello,dry,3,2,3,0
9,2020,10,sochi,dry,2,0,1,1


In [ ]:
def features(results):

    # Ordena el DataFrame por temporada, ronda y cualquier otro criterio necesario
    results = results.sort_values(by=['season', 'round'], ascending=[True, True])

    # Calcula los puntos acumulados por piloto para cada temporada
    results['DriverPointsCumulative'] = results.groupby(['season', 'DriverId'])['Points'].cumsum()

    # Calcula los puntos acumulados por equipo para cada temporada
    results['TeamPointsCumulative'] = results.groupby(['season', 'TeamId'])['Points'].cumsum()

    # Visualiza las primeras filas para verificar
    print(results[['season', 'round', 'DriverId', 'TeamId', 'Points', 'DriverPointsCumulative', 'TeamPointsCumulative']].head())


In [88]:
results_df

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,weather,yellows,reds,safety_cars,virtual_safety_cars,season,round,circuitId
77,bottas,mercedes,1.0,1.0,0.000,Finished,25.0,dry,7,0,3,0,2020,1,red_bull_ring
16,leclerc,ferrari,2.0,7.0,2.700,Finished,18.0,dry,7,0,3,0,2020,1,red_bull_ring
4,norris,mclaren,3.0,3.0,5.491,Finished,16.0,dry,7,0,3,0,2020,1,red_bull_ring
44,hamilton,mercedes,4.0,5.0,5.689,Finished,12.0,dry,7,0,3,0,2020,1,red_bull_ring
55,sainz,mclaren,5.0,8.0,8.903,Finished,10.0,dry,7,0,3,0,2020,1,red_bull_ring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,giovinazzi,alfa,16.0,14.0,202.738,+1 Lap,0.0,dry,1,0,1,1,2020,17,yas_marina
6,latifi,williams,17.0,18.0,202.738,+1 Lap,0.0,dry,1,0,1,1,2020,17,yas_marina
20,kevin_magnussen,haas,18.0,20.0,202.738,+1 Lap,0.0,dry,1,0,1,1,2020,17,yas_marina
51,pietro_fittipaldi,haas,19.0,17.0,202.738,+2 Laps,0.0,dry,1,0,1,1,2020,17,yas_marina


In [89]:
# Ordenar para sumar bien
results_df = results_df.sort_values(by='round', ascending=True)

# Agrupa los resultados por piloto y calcula los puntos acumulados
results_df['DriverPointsCumulative'] = results_df.groupby('DriverId')['Points'].cumsum()

# Agrupa los resultados por equipo y calcula los puntos acumulados
results_df['TeamPointsCumulative'] = results_df.groupby('TeamId')['Points'].cumsum()

# Visualiza las primeras filas para verificar
results_df

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,weather,yellows,reds,safety_cars,virtual_safety_cars,season,round,circuitId,DriverPointsCumulative,TeamPointsCumulative
77,bottas,mercedes,1.0,1.0,0.000,Finished,25.0,dry,7,0,3,0,2020,1,red_bull_ring,25.0,25.0
33,max_verstappen,red_bull,20.0,2.0,131.650,Electronics,0.0,dry,7,0,3,0,2020,1,red_bull_ring,0.0,0.0
3,ricciardo,renault,19.0,10.0,131.650,Overheating,0.0,dry,7,0,3,0,2020,1,red_bull_ring,0.0,0.0
18,stroll,racing_point,18.0,9.0,131.650,Engine,0.0,dry,7,0,3,0,2020,1,red_bull_ring,0.0,0.0
20,kevin_magnussen,haas,17.0,16.0,131.650,Brakes,0.0,dry,7,0,3,0,2020,1,red_bull_ring,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,bottas,mercedes,2.0,2.0,15.976,Finished,18.0,dry,1,0,1,1,2020,17,yas_marina,223.0,573.0
33,max_verstappen,red_bull,1.0,1.0,0.000,Finished,25.0,dry,1,0,1,1,2020,17,yas_marina,214.0,319.0
51,pietro_fittipaldi,haas,19.0,17.0,202.738,+2 Laps,0.0,dry,1,0,1,1,2020,17,yas_marina,0.0,3.0
31,ocon,renault,9.0,10.0,101.069,Finished,2.0,dry,1,0,1,1,2020,17,yas_marina,62.0,181.0


In [95]:
results_df

,DriverId,TeamId,Position,GridPosition,Time,Status,Points,weather,yellows,reds,safety_cars,virtual_safety_cars,season,round,circuitId,DriverPointsCumulative,TeamPointsCumulative
77,bottas,mercedes,1.0,1.0,0.000,Finished,25.0,dry,7,0,3,0,2020,1,red_bull_ring,25.0,25.0
33,max_verstappen,red_bull,20.0,2.0,131.650,Electronics,0.0,dry,7,0,3,0,2020,1,red_bull_ring,0.0,0.0
3,ricciardo,renault,19.0,10.0,131.650,Overheating,0.0,dry,7,0,3,0,2020,1,red_bull_ring,0.0,0.0
18,stroll,racing_point,18.0,9.0,131.650,Engine,0.0,dry,7,0,3,0,2020,1,red_bull_ring,0.0,0.0
20,kevin_magnussen,haas,17.0,16.0,131.650,Brakes,0.0,dry,7,0,3,0,2020,1,red_bull_ring,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,bottas,mercedes,2.0,2.0,15.976,Finished,18.0,dry,1,0,1,1,2020,17,yas_marina,223.0,573.0
33,max_verstappen,red_bull,1.0,1.0,0.000,Finished,25.0,dry,1,0,1,1,2020,17,yas_marina,214.0,319.0
51,pietro_fittipaldi,haas,19.0,17.0,202.738,+2 Laps,0.0,dry,1,0,1,1,2020,17,yas_marina,0.0,3.0
31,ocon,renault,9.0,10.0,101.069,Finished,2.0,dry,1,0,1,1,2020,17,yas_marina,62.0,181.0
